In [14]:
import requests
import pandas as pd
from datetime import datetime
import time
import os

def fetch_all_news(symbol, start_date, end_date, api_token):
    """
    Fetch all news articles for a symbol between two dates
    """
    all_news = []
    offset = 0
    limit = 1000

    url = 'https://eodhd.com/api/news'

    while True:
        params = {
            'api_token': api_token,
            's': symbol,
            'from': start_date,
            'to': end_date,
            'offset': offset,
            'limit': limit
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            news_batch = response.json()

            if not news_batch or len(news_batch) == 0:
                break

            all_news.extend(news_batch)

            if len(news_batch) < limit:
                break

            offset += limit
            time.sleep(0.5)

        except requests.exceptions.RequestException as e:
            print(f"  ❌ Error fetching {symbol}: {e}")
            break

    return all_news

def process_stock_news(ticker, symbol, start_date, end_date, api_token):
    """
    Process news for a single stock and return DataFrame with 3 columns
    """
    print(f"  Processing {ticker}...", end=" ")

    news = fetch_all_news(symbol, start_date, end_date, api_token)

    if not news:
        print(f"No news found")
        return None

    df = pd.DataFrame(news)
    df['date'] = pd.to_datetime(df['date'])
    df['date_only'] = df['date'].dt.date
    df['published_time'] = df['date'].dt.strftime('%H:%M:%S')

    df['ticker'] = ticker

    print(f"✅ {len(df)} articles")

    def format_news_for_day(group):
        entries = []
        for _, row in group.iterrows():
            entry = f"⏰ {row['published_time']}\n📰 {row['title']}\n📄 {row.get('content', 'No content')}\n🔗 {row.get('link', '')}"
            entries.append(entry)
        return '\n\n' + ('-'*100 + '\n\n').join(entries)


    daily_news = df.groupby(['date_only', 'ticker']).apply(format_news_for_day).reset_index()
    daily_news.columns = ['Date', 'Ticker', 'News']
    daily_news = daily_news.sort_values('Date')

    return daily_news


# The API is personal, so I didn't put my API here,
# You can you your personal Eodhd API ro run this code
API_TOKEN = 'This is Personal API'
START_DATE = '2023-01-01'
END_DATE = '2023-12-31'
OUTPUT_DIR = 'stock_news_2023'


os.makedirs(OUTPUT_DIR, exist_ok=True)


STOCKS = [
    'AAPL', 'AMZN', 'ASML', 'AVGO', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA',
    'PYPL',
    'COST', 'EBAY', 'JD', 'PDD', 'NFLX',
    'ENPH',
    'TMUS', 'FAST', 'MRNA', 'CSCO'
]

print(f"🚀 Fetching news for {len(STOCKS)} stocks")
print(f"📅 Period: {START_DATE} to {END_DATE}\n")

all_data = []

for i, ticker in enumerate(STOCKS, 1):
    print(f"[{i}/{len(STOCKS)}]", end=" ")
    symbol = f"{ticker}.US"

    try:
        df = process_stock_news(ticker, symbol, START_DATE, END_DATE, API_TOKEN)
        if df is not None:
            all_data.append(df)
    except Exception as e:
        print(f"❌ Error: {e}")

    if i < len(STOCKS):
        time.sleep(1)

# Combine all stocks into one dataset
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values(['Date', 'Ticker'])

    output_csv = os.path.join(OUTPUT_DIR, 'all_stocks_daily_news_2023.csv')
    combined_df.to_csv(output_csv, index=False)


    print(f"\n{'='*60}")
    print(f"📊 SUMMARY")
    print(f"{'='*60}")
    print(f"✅ Total rows: {len(combined_df)}")
    print(f"📅 Date range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
    print(f"💾 Saved to:")
    print(f"   - {output_csv}")
    print(f"{'='*60}")

    # Show sample
    print(f"\n📋 Sample data:")
    print(combined_df.head(3))
else:
    print("\n❌ No data retrieved")

🚀 Fetching news for 20 stocks
📅 Period: 2023-01-12 to 2023-01-13

[1/20]   Processing AAPL... ✅ 50 articles


/tmp/ipython-input-8439503.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_news = df.groupby(['date_only', 'ticker']).apply(format_news_for_day).reset_index()


[2/20]   Processing AMZN... ✅ 46 articles


/tmp/ipython-input-8439503.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_news = df.groupby(['date_only', 'ticker']).apply(format_news_for_day).reset_index()


[3/20]   Processing ASML... ✅ 4 articles


...

/tmp/ipython-input-8439503.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_news = df.groupby(['date_only', 'ticker']).apply(format_news_for_day).reset_index()


[19/20]   Processing MRNA... ✅ 8 articles


/tmp/ipython-input-8439503.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_news = df.groupby(['date_only', 'ticker']).apply(format_news_for_day).reset_index()


[20/20]   Processing CSCO... ✅ 1 articles

📊 SUMMARY
✅ Total rows: 35
📅 Date range: 2023-01-12 to 2023-01-13
💾 Saved to:
   - stock_news_2023/all_stocks_daily_news_2023.csv

📋 Sample data:
         Date Ticker                                               News
0  2023-01-12   AAPL  \n\n⏰ 23:50:37\n📰 Apple CEO Cook's pay more de...
2  2023-01-12   AMZN  \n\n⏰ 21:58:22\n📰 23 Easy Ways To Save Money T...
4  2023-01-12   ASML  \n\n⏰ 22:29:06\n📰 UPDATE 1-U.S. talks with Jap...


/tmp/ipython-input-8439503.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_news = df.groupby(['date_only', 'ticker']).apply(format_news_for_day).reset_index()
